# Cálculo de NDVI

En este ejercicio vas a trabajar con imágenes satelitales de alta calidad de **Sentinel-2** para visualizar las bandas **Roja (R)** y **Infrarrojo Cercano (NIR)** de una zona relativamente pequeña (del orden de $100\\text{ m}^2$). A partir de esas bandas vas a calcular y analizar el **NDVI (Normalized Difference Vegetation Index)**, un índice muy usado para estudiar el estado de la vegetación.

Antes del taller, completá la primera sección (**Configuración**).

## Configuración

0. Create un Environment

        conda create -n geo_env python=3.11 rasterio numpy matplotlib jupyterlab -c conda-forge
        
1. Asegurate de tener tu directorio de proyecto ordenado:

        project-directory/
        │
        ├── images/
        │
        ├── raw/
        │   ├── ndvi/
        │   ├── ndwi/
        │   ├── time_series_1/
        │   └── time_series_2/
        │
        └── script.py

   Vamos a usar la carpeta `raw/` para guardar los archivos ráster (imágenes TIFF de Sentinel).

2. Verificá que las librerías necesarias estén instaladas y configuradas en tu entorno de Python.

In [ ]:
# Install required packages (uncomment to install if needed)
# !pip install rasterio numpy matplotlib
# Import libraries
import rasterio
import numpy as np
from pathlib import Path
import matplotlib.pyplot as plt
import re

# Print versions of the libraries to ensure compatibility
print(f"Rasterio version: {rasterio.__version__}")
print(f"Numpy version: {np.__version__}")

Documentación, ejemplos y explicaciones para cada librería que vas a usar:

- [**rasterio**](https://rasterio.readthedocs.io/): Para leer y procesar datos ráster.
- [**numpy**](https://numpy.org/doc/): Para operaciones numéricas eficientes.
- [**pathlib**](https://docs.python.org/3/library/pathlib.html): Para manejar rutas de archivos de forma cómoda y moderna.
- [**matplotlib**](https://matplotlib.org/stable/contents.html): Para visualizar datos e imágenes.
- [**re (Expresiones regulares)**](https://docs.python.org/3/library/re.html): Para trabajar con patrones de texto.

> Tip: no hace falta leerse toda la documentación de un saque; la idea es saber que está ahí cuando la necesites (como ese cajón desordenado de la cocina que siempre termina salvando la noche).


## Guía paso a paso para el análisis geoespacial

### Paso 1: Acceso a imágenes del Sentinel

- La **[SentiWiki](https://sentiwiki.copernicus.eu/web/sentiwiki)** tiene información de los satélites, los diseños y los datos que adquieren. 
- Acceso a los datos del Sentinel: [Copernicus Open Access Hub](https://dataspace.copernicus.eu/).

En esta parte vas a aprender a acceder a imágenes de Sentinel para una región concreta. Podés usar coordenadas (latitud y longitud) para definir el área de interés (AOI) y descargar las bandas que te hagan falta.

**¿Qué es lo que realmente estás abriendo? Un archivo Raster.**

Una imagen satelital se puede pensar como una grilla (matriz) donde cada píxel representa un pequeño pedazo de la superficie terrestre y almacena un valor de reflectancia en una determinada banda del espectro (Rojo, NIR, Azul, Verde, etc.).

Los archivos `.tiff` (GeoTIFF) suelen incluir además metadatos importantes:
- Sistema de referencia de coordenadas.
- Resolución espacial.
- Número de bandas.
- Información adicional sobre el sensor y el producto.

La idea es que entiendas qué estás manipulando antes de tirarle operaciones encima.

> Si todo esto te parece mucha teoría, quedate con la idea básica: “cada píxel es un numerito” y después ya le vamos agregando filosofía.


### Paso 2: Descarga de bandas

Identificá y descargá las bandas necesarias para calcular el **NDVI**. En Sentinel-2, típicamente se usa:

- Banda **Roja** (B4).
- Banda **NIR** (Infrarrojo Cercano, B8).

Al descargar:

- Definí la ubicación exacta usando coordenadas (lat/long) o un polígono.
- Elegí el formato **TIFF** con profundidad de **16 bits**.
- Usá una resolución adecuada (por ejemplo, una opción *MEDIUM* puede ser suficiente para este ejercicio).
- Podés usar `pathlib` para manejar rutas y mover/copiar archivos de forma ordenada.

**¿Qué es un píxel (“px”)?**  
Es la unidad mínima de la imagen. Cada píxel cubre un área en el terreno (por ejemplo, 10 m x 10 m para ciertas bandas de Sentinel-2) y guarda un valor de reflectancia.

> Cada píxel es un “terrenito” donde el satélite midió cuánta luz se reflejó. Algunos son verdes y felices, otros están más secos que yerba secándose al sol.

Cuando uses `rasterio.open(...)`, fijate en atributos como:

- `type(src).__name__`
- `src.width`, `src.height`
- `src.crs` (sistema de coordenadas)
- `src.count` (número de bandas)
- `src.meta` (diccionario con metadatos)
- la forma de `src.read()` y `src.read(1)`


In [ ]:
# Get data from Sentinel-2 L2A
directory_path = Path('raw/ndvi')
# Gey TIFF files names
for file_path in directory_path.glob("*.tiff"):
    print(file_path)
print('-'*20)

for file_path in directory_path.glob("*B*.tiff"):
    print('Choose a file:')
    print(file_path)
    file_path = file_path
print('*'*20)

# Type the solution here:
# ............................


### Paso 3: Explorar, caracterizar y visualizar los datos

Primero vas a definir una función que lea una banda desde un archivo `.tiff` y devuelva:

- Los datos de la banda como un array de `numpy`.
- Los metadatos asociados (por ejemplo, el perfil de `rasterio`).

Usá **rasterio**, una librería pensada para trabajar con ráster geoespaciales, y combinála con **numpy** para las operaciones numéricas. Convertí los datos a `float32`, ya que lo vas a necesitar para los cálculos posteriores.

Una vez implementada la función:

1. Probala leyendo una banda.
2. Inspeccioná el resultado usando:
   - `.min()`
   - `.max()`
   - `.shape`

Así te das una idea del rango de valores y del tamaño de la imagen.

Finalmente, visualizá las bandas. Podés usar `plt.imshow(...)` con un mapa de colores `cmap="RdYlGn"` u otro que te guste. Mostrá las imágenes una al lado de la otra, activá los ejes (`plt.axis("on")` o similar) y tratá de interpretar qué ves en cada banda.

> Bonus filosófico: cuando activás los ejes y ves las coordenadas, el mapa deja de ser “una fotito linda” y pasa a ser “un dato científico”. Ahí ya te convertiste oficialmente en persona seria.


In [ ]:
def read_band(filename):
    """
    Reads a the first band of the file from a TIFF file.
    - Parameters:
        file_path (Path or str): Full path to the file.
    - Returns: tuple, (band_data, metadata)
        band_data (numpy.ndarray): NumPy array of the band values, converted to float32.
        metadata (dict): Metadata of the file.
    """

    # Type the solution here:
    #........................


In [ ]:
# Try the function here:
#.......................

In [ ]:
directory_path = Path('raw/ndvi')

for file_path in directory_path.glob("*.tiff"):
    print(file_path.name)

    if "B08" in file_path.name:  # Check if the file corresponds to B08
        B08, _ = read_band(file_path)
        
        # Complete the code below:
        #.........................
        print(f"- Band shape:", ##COMPLETE THE CODE HERE##)
        print('- B8 max:', ##COMPLETE THE CODE HERE##)
        print('- B8 min:', ##COMPLETE THE CODE HERE##) 
        print('')

    elif "B04" in file_path.name:  # Check if the file corresponds to B04
        B04, _ = read_band(file_path)
        
        # Complete the code below:
        #.........................
        print(f"Band shape:", ##COMPLETE THE CODE HERE##)
        print('- B4 max:', ##COMPLETE THE CODE HERE##)
        print('- B4 min:', ##COMPLETE THE CODE HERE##)


In [ ]:
# Visualize both bands side by side
# Type the solution here:
# .......................


### Nota sobre métricas e interpretación (para más adelante)

Más adelante en el notebook vas a trabajar con series temporales de NDVI (varios años). Allí vas a:

- Calcular diferentes métricas para cada imagen (media, mediana, porcentaje de alta vegetación, etc.).
- Compilar esas métricas en un pequeño conjunto de datos.
- Representar la evolución en el tiempo con un gráfico de líneas (eje X: años, eje Y: métricas de NDVI).

Las preguntas clave serán cosas como:

- ¿Cómo cambió la vegetación entre 2016 y 2024?
- ¿Se ve recuperación, degradación o patrones estacionales?
- ¿Qué dicen las métricas sobre el “estado de salud” del área?

> Podés saltearte esta nota por ahora y volver cuando llegues a la parte de series temporales. No hace falta adelantarse… salvo que seas de esas personas que leen el final del libro primero.


# ![Espectro de luz](images/Light_Spectrum.png)

Las bandas suelen representar valores de reflectancia que fueron **re-escalados** a un cierto rango (por ejemplo, entre 0 y un máximo entero).

Lo importante es entender qué parte del espectro cubre cada banda (visible, infrarrojo, etc.) y cómo responde la vegetación o el agua en esas longitudes de onda.

> En criollo: distintas cosas en la superficie reflejan la luz de manera distinta. A la vegetación le gusta el NIR, al agua no tanto.


### Paso 4: Cálculo de NDVI

El **NDVI (Normalized Difference Vegetation Index)** es un índice muy usado en teledetección para analizar la cobertura y el estado de la vegetación.

Se define a partir de la reflectancia en la banda **Roja (RED)** y en la banda **Infrarrojo Cercano (NIR)**. En términos simples:

- La vegetación sana absorbe mucha luz roja y refleja bastante NIR.
- Superficies sin vegetación (suelo desnudo, construcciones) tienden a reflejar más rojo y menos NIR.

#### Fórmula

El NDVI se calcula como:

$$\text{NDVI} = \frac{\text{NIR} - \text{RED}}{\text{NIR} + \text{RED}} \qquad\qquad(1)$$

Los valores resultantes suelen ir de -1 a 1 (aunque en la práctica el rango útil suele estar más concentrado).

#### Actividades

Con las bandas que ya cargaste y exploraste:

1. **Evitá divisiones por cero:** asegurate de que $\text{NIR} + \text{RED} \neq 0$ antes de aplicar la fórmula.
2. **Calculá el NDVI:** usá arrays de `numpy` para implementar la fórmula (1).
3. **Caracterizá el NDVI:** imprimí valores mínimos, máximos y la forma (`shape`) del array de NDVI.
4. **Visualizá el NDVI:** generá una figura para inspeccionar el patrón espacial del índice.
5. Probá visualizarlo **con** y **sin** fijar la escala usando `vmin=-1, vmax=1`. ¿Qué cambia en la interpretación del mapa?

> Si el mapa te sale todo rojo o todo verde, seguramente no es culpa del satélite: revisá el código antes de putear al hardware.


In [ ]:
# Check if any values of nir + red are 0
sum_nir_red =  # Perform the addition

# Complete the code below:
#.........................
zeros_exist =  # Check if any value is 0

print(zeros_exist)

In [ ]:
# Calculate & Plot NDVI
# Type the solution here:
# .......................


### Paso 5: Índices adicionales

Además del NDVI, se pueden construir otros índices a partir de las mismas bandas o de combinaciones distintas. Uno muy usado para cuerpos de agua es el **NDWI (Normalized Difference Water Index)**.

El NDWI busca resaltar zonas con agua y atenuar la presencia de vegetación y suelo. Se calcula usando la banda **Verde (B03)** y la banda **NIR (B08)**:

$$\text{NDWI} = \frac{\text{Green} - \text{NIR}}{\text{Green} + \text{NIR}} \qquad\qquad(2)$$

En esta fórmula, el agua suele tener:
- Baja reflectancia en NIR (B08).
- Mayor reflectancia en la banda verde (B03).

Los valores de NDWI también van, en teoría, de -1 a 1. Valores positivos tienden a indicar presencia de agua; valores negativos suelen corresponder a suelo, vegetación o áreas urbanas.

> Resumen rápido: NDVI ama la vegetación, NDWI ama el agua. Como en la vida, no siempre se puede quedar bien con los dos a la vez.


In [ ]:
def store_images(file_path, images):
    """
    Reads and stores image data and metadata into a dictionary.

    Parameters:
        file_path (Path): Full path to the image file.
        images (dict): Dictionary to store image data.

    Returns:
        None: Updates the 'images' dictionary in place.
    """

    images[file_path] = {
            "data": # Read data. 
            "metadata": # Read metadata. You have a function for doing both :)
            "date": # Extract the date from file_path.name
        }
    
    print(f"Loaded {file_path.name} with date: {images[file_path]['date']}")

In [ ]:
# Path to the directory containing the images
directory_path = Path('raw/ndwi')

# Dictionary to store loaded images
images = {}

# Iterate over the files in the directory
for file_path in directory_path.glob("*.tiff"):
    # Complete the code below:
    #.........................


# Check contents of the dictionary
# Iterate over the key-values
for ... in ...:
    print(f"\nImage Name: {##COMPLETE THE CODE HERE##.stem}")
    print(f"Data Shape: {##COMPLETE THE CODE HERE##}")
    print(f"Scope: [Min = {##COMPLETE THE CODE HERE##}, Max = {##COMPLETE THE CODE HERE##}]")

In [ ]:
# Plot NDVI for both images
# Type the solution here:
# .......................

### Paso 6: Operaciones sobre imágenes

#### Umbralización de NDWI para aislar cuerpos de agua

Con el NDWI calculado, podés identificar píxeles que corresponden (aproximadamente) a agua aplicando un **umbral**.

1. Analizá el rango de valores de NDWI (mínimo, máximo, histograma).
2. Usá código similar al provisto para entender cómo funcionan las **máscaras booleanas**. Por ejemplo, si tenés un array `[0.15, 0.05, 0.30, 0.20]` y elegís `threshold = 0.15`, la comparación `array > threshold` da `[False, False, True, True]`.
3. Implementá una función que, dado un NDWI y un umbral, devuelva una máscara binaria (agua / no agua).
4. Visualizá los resultados en varias figuras (por ejemplo, imagen original, NDWI, máscara, NDWI enmascarado).

Preguntas clave:

- ¿Qué pasa cuando bajás o subís el valor del umbral?
- ¿Aparecen zonas mal clasificadas (por ejemplo, áreas que no son agua pero entran como “agua”)?

> Spoiler: elegir el umbral “perfecto” es más arte que ciencia. Pero no lo digas en voz alta delante de alguien que hace clasificación supervisada.


In [ ]:
ndvi_real = np.array([0.15, 0.05, 0.30, 0.20])
# Complete the code below:
#.........................
mask = ##COMPLETE THE CODE HERE##
print(mask)

In [ ]:
def apply_mask(data, threshold=1000):
    """
    Applies a mask to the data based on a threshold.

    Parameters:
        data (numpy.ndarray): The NDWI data to mask.
        threshold (float): The value above which pixels are considered water.

    Returns:
        numpy.ndarray: Binary mask with True for values above the threshold.
    """
    # Complete the code below:
    #.........................

In [ ]:
# Create a 2x2 grid for visualization
# NDVI for both dates, masked NDVI for both dates:
#.........................

#### Sustracción de imágenes para detección de cambios

En teledetección, comparar imágenes de distintos momentos es una forma poderosa de detectar cambios en lagos, embalses u otras superficies de agua.

Si tenés NDWI para dos fechas (por ejemplo, un año húmedo y un año seco), podés restar:

$$\text{Cambio} = \text{NDWI}{(\text{fecha tardía})} - \text{NDWI}{(\text{fecha temprana})}$$

- Valores **positivos** indican un aumento en el NDWI (posible incremento en agua).
- Valores **negativos** indican disminución (posible retracción del cuerpo de agua).

La imagen resultante resalta las zonas donde cambió el nivel de agua y te permite cuantificar cuánto se redujo o expandió el embalse/reservorio.

**Pregunta extra:**  
¿Podés restar **dos imágenes enmascaradas** (por ejemplo, donde solo se ve el agua)? ¿Qué esperarías ver en ese caso?

> Consejo práctico: antes de sacar conclusiones dramáticas sobre sequías históricas, fijate bien que no te hayas confundido de año o de archivo. Pasa más seguido de lo que parece.


In [ ]:
# Type the code here:
#...................

### Paso 7: Análisis de NDVI para múltiples archivos (serie temporal)

En esta última parte vas a trabajar con **varias imágenes** de NDVI correspondientes a distintos años (por ejemplo, 2016–2024) para estudiar la evolución de la vegetación en el área de interés.

1. Empezá con una sola imagen:
    - Abrí y visualizá el **NDVI** de un archivo.
    - Ajustá los valores al intervalo [-1, 1], mapeando correctamente el rango disponible.
    - Mostrá la imagen de NDVI “cruda” y la normalizada lado a lado.
    - Calculá algunas métricas:
        - **NDVI medio (mean):** nivel promedio de vegetación.
        - **NDVI mediano (median):** más robusto frente a valores extremos.
        - **Porcentaje de alta vegetación:** proporción de píxeles con NDVI por encima de un cierto umbral.

2. Extendé el análisis a todos los años disponibles:
    - Repetí el cálculo de métricas para cada imagen de NDVI.
    - Guardá estas métricas en una estructura tipo tabla (por ejemplo, un `DataFrame`).
    - Creá un gráfico de líneas donde:
        - Eje X: años (2016–2024).
        - Eje Y: métricas de NDVI (media, mediana, porcentaje de alta vegetación).
    - Analizá las tendencias temporales en la salud de la vegetación.

3. Preguntas clave:
    - ¿Cómo evolucionó la vegetación entre 2016 y 2024?
    - ¿Se observan tendencias claras (recuperación, degradación, estacionalidad)?
    - ¿Qué cuentan las métricas sobre el estado general del área?

> Si llegaste hasta acá sin que se rompa nada, podés premiarte con un mate o algo rico: ya estás haciendo análisis de series temporales con imágenes satelitales como si nada.


In [ ]:
# Mask to copmute only pixel above the threshold:
# Understand this technique:
ndvi_real = np.array([[0.15, 0.05, 0.30, 0.20],
                      [0.10, 0.35, 0.20, 0.20]])
print('Array:\n', ndvi_real)
mask = ##COMPLETE THE CODE HERE##
print('Mask:\n', mask)
filtered_values = ndvi_real[mask]
print('Filtered values:', filtered_values)
print('Filtered values sum:', filtered_values.sum())
print('Filtered values mean:', filtered_values.mean())

In [ ]:
# Function to normalize NDVI
def normalize_ndvi(ndvi_raw):
    return -(ndvi_raw - ndvi_raw.max() / 2) / (ndvi_raw.max() / 2)

# Function to calculate NDVI metrics
def calculate_metrics(ndvi_normalized, thresholds=[0.5, 0.6, 0.7]):
    mean_ndvi = ndvi_normalized.mean()
    median_ndvi = np.median(ndvi_normalized)
    high_veg_percentages = [
        (ndvi_normalized > threshold).sum() / ndvi_normalized.size * 100
        for threshold in thresholds
    ]
    return mean_ndvi, median_ndvi, high_veg_percentages

In [ ]:
# Root directory
directory_path = Path('raw/time_series_1')

for i, file_path in enumerate(sorted(directory_path.glob("*2024*.tiff"))):
    ndvi_raw, _ = read_band(file_path)

    print(f"Image Shape: {file_path.name}")
    print(f"Data Shape: {ndvi_raw.shape}")
    print(f"Scope: Min = {ndvi_raw.min()}, Max = {ndvi_raw.max()}\n")

    ndvi_normalized = normalize_ndvi(ndvi_raw)
    print(f"Normalized Data Shape: {ndvi_normalized.shape}")
    print(f"Scope: Min = {ndvi_normalized.min()}, Max = {ndvi_normalized.max()}")

    # Calculate metrics for this image
    thresholds=[0.4, 0.6, 0.7]
    mean_ndvi, median_ndvi, high_veg = calculate_metrics(ndvi_normalized, thresholds)

    # Print metrics
    print(f"Mean NDVI: {mean_ndvi:.4f}")
    print(f"Median NDVI: {median_ndvi:.4f}")
    print(f"High Vegetation Percentage (NDVI > {thresholds[0]}): {high_veg[0]:.2f}%")
    print(f"High Vegetation Percentage (NDVI > {thresholds[1]}): {high_veg[1]:.2f}%")
    print(f"High Vegetation Percentage (NDVI > {thresholds[2]}): {high_veg[2]:.2f}%")

    # Plot both images side by side
    plt.figure(figsize=(12, 4))

    # Plot raw NDVI
    plt.subplot(1, 2, 1)
    plt.title("Raw NDVI")
    plt.imshow(ndvi_raw, cmap='RdYlGn')
    plt.colorbar(label='Raw NDVI Value')
    plt.axis('off')

    # Plot normalized NDVI
    plt.subplot(1, 2, 2)
    plt.title("Normalized NDVI (-1 to 1)")
    plt.imshow(ndvi_normalized, cmap='RdYlGn')
    plt.colorbar(label='Normalized NDVI Value')
    plt.axis('off')

    # Display the plots
    plt.tight_layout()
    plt.show()

In [ ]:
# Variables to store NDVI mean values
ndvi_means = []
ndvi_means_vegetation = {threshold: [] for threshold in [0.1, 0.3, 0.6]}  # NDVI vegetation means by threshold
years = []

# Configure the 3x3 grid for plotting NDVI images
fig, axes = plt.subplots(3, 3, figsize=(15, 15))
axes = axes.flatten()

# Process all .tiff files in the directory
for i, file_path in enumerate(sorted(directory_path.glob("*.tiff"))):
    # Extract the year from the filename
    year = int(re.search(r"(\d{4})", file_path.name).group(1))
    years.append(year)
    
    # Open NDVI file
    band, _ = read_band(file_path)
    # Descale NDVI
    ndvi_real = (32768 - band) / 32768
    ndvi_real = normalize_ndvi(band)
    # Calculate overall NDVI mean
    ndvi_means.append(ndvi_real.mean())
    # Calculate NDVI means for vegetation thresholds
    for threshold in ndvi_means_vegetation.keys():
        mask = ndvi_real > threshold
        ndvi_means_vegetation[threshold].append(ndvi_real[mask].mean())

    # Plot each NDVI image in the 3x3 grid
    ax = axes[i]
    im = ax.imshow(ndvi_real, cmap='RdYlGn', vmin=-1, vmax=1)
    ax.set_title(f"Year: {year}", fontsize=10)
    ax.axis('off')

# Add color bar below the grid
cbar = fig.colorbar(im, ax=axes, orientation='horizontal', fraction=0.03, pad=0.07)  # Reduce padding
cbar.set_label('NDVI Value', fontsize=12)

# Adjust layout
plt.suptitle('Monitoring Vegetation Dynamics Through Temporal NDVI Analysis (2016–2024)', fontsize=16, y=0.75)
plt.subplots_adjust(top=0.7, bottom=0.15, wspace=0.1, hspace=0.2)
plt.show()

In [ ]:
# Plot NDVIs averages vs. year
plt.figure(figsize=(10, 6))
plt.plot(years, ndvi_means, marker='o', label='NDVI Mean', color='blue')  # NDVI Promedio

# Add NDVI means for each threshold to the plot
colors = ['green', 'orange', 'red']  # Colors for each threshold
for threshold, color in zip(ndvi_means_vegetation.keys(), colors):
    plt.plot(years, ndvi_means_vegetation[threshold], marker='o', label=f'NDVI Vegetation (> {threshold})', color=color)

plt.xlabel('Year')
plt.ylabel('NDVI Mean')
plt.title('NDVI Mean and NDVI Vegetation Mean vs. Year')
plt.legend()
plt.grid()
plt.show()

In [ ]:
# Function to calculate NDVI metrics with thresholds
def calculate_threshold_metrics(ndvi_normalized, thresholds=[0.1, 0.3, 0.6]):
    """
    Calculate mean NDVI for pixels above each threshold.

    Parameters:
        ndvi_normalized (numpy.ndarray): Normalized NDVI array.
        thresholds (list): List of thresholds for NDVI ranges.

    Returns:
        list: Mean NDVI values for each threshold.
    """
    threshold_means = [
        ndvi_normalized[ndvi_normalized > threshold].mean() if (ndvi_normalized > threshold).any() else 0
        for threshold in thresholds
    ]
    return threshold_means

# Path to directory containing NDVI images
directory_path = Path('raw/time_series_2')

# Read and process NDVI images
ndvi_data = []
metrics = []
threshold_metrics = []
dates = []

thresholds = [0.1, 0.3, 0.6]

for file_path in sorted(directory_path.glob("*.tiff")):
    ndvi_raw, _ = read_band(file_path)
    ndvi_normalized = normalize_ndvi(ndvi_raw)
    ndvi_data.append(ndvi_normalized)

    # Calculate overall mean and median NDVI
    mean_ndvi = ndvi_normalized.mean()
    median_ndvi = np.median(ndvi_normalized)

    # Calculate mean NDVI for pixels above thresholds
    threshold_means = calculate_threshold_metrics(ndvi_normalized, thresholds)

    metrics.append((mean_ndvi, median_ndvi))
    threshold_metrics.append(threshold_means)

    # Extract date from filename
    date = file_path.stem.split("_")[0]  # Adjust based on filename format
    dates.append(date)

# Convert metrics to NumPy arrays for easier processing
metrics = np.array(metrics)
threshold_metrics = np.array(threshold_metrics).T  # Transpose for thresholds
mean_ndvi_values, median_ndvi_values = metrics.T

# Plot NDVI images in a 3x3 grid with a single color bar
fig, axes = plt.subplots(3, 3, figsize=(15, 9), constrained_layout=True)

# Normalize color bar limits
vmin, vmax = -1, 1  # NDVI range
cmap = 'RdYlGn'

for i, (ax, ndvi_normalized) in enumerate(zip(axes.ravel(), ndvi_data)):
    im = ax.imshow(ndvi_normalized, cmap=cmap, vmin=vmin, vmax=vmax)
    ax.set_title(f"NDVI {dates[i]}")
    ax.axis('off')

# Add a single color bar for the entire grid
fig.colorbar(im, ax=axes, orientation='vertical', shrink=0.8, label='NDVI Value')

# Add a title for the whole figure
fig.suptitle("Normalized NDVI (2016-2024)", fontsize=16)
plt.show()

# Plot temporal trends in NDVI metrics
plt.figure(figsize=(12, 6))

# Plot mean NDVI
plt.plot(dates, mean_ndvi_values, label="Mean NDVI", marker="o")

# Plot median NDVI
plt.plot(dates, median_ndvi_values, label="Median NDVI", marker="s")

# Plot Mean NDVI for thresholds
for i, threshold in enumerate(thresholds):
    plt.plot(
        dates,
        threshold_metrics[i],
        label=f"Mean NDVI (NDVI > {threshold})",
        marker="^",
    )

plt.xlabel("Year")
plt.ylabel("NDVI Value")
plt.title("Temporal Trends in NDVI Metrics (2016-2024)")
plt.legend()
plt.grid(True)
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()